# 数据集扩充

虽然当前的模型已经能够达到较好的效果，但是还不够好，对于一些较老的烟梗不能够做到有效的判别，我们为此增加数据集。

In [1]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np

from utils import read_raw_file, split_xy, generate_tobacco_label, generate_impurity_label
from models import SpecDetector
import pickle

In [2]:
# some parameters
new_spectra_file = r"F:\zhouchao\618\bomo\calibrated7.raw"
new_label_file = r"F:\zhouchao\618\bomo\picture\label7.bmp"

target_class = 0
target_class_left, target_class_right = 5, 4
light_threshold = 0.5
add_background = False

split_line = 500


blk_sz, sensitivity = 8, 8
selected_bands = [127, 201, 202, 294]
tree_num = 185

pic_row, pic_col= 600, 1024

color_dict = {(0, 0, 255): 1, (255, 255, 255): 0, (0, 255, 0): 2, (255, 0, 0): 1, (0, 255, 255): 4,
          (255, 255, 0): 5, (255, 0, 255): 6}

new_dataset_file = f'./dataset/data_{blk_sz}x{blk_sz}_c{len(selected_bands)}_sen{sensitivity}_7.p'
dataset_file = f'./dataset/data_{blk_sz}x{blk_sz}_c{len(selected_bands)}_sen{sensitivity}_6.p'

model_file = f'./models/rf_{blk_sz}x{blk_sz}_c{len(selected_bands)}_{tree_num}_6.model'
selected_bands = None

In [3]:
# if len(new_spectra_files) == 1:
data = read_raw_file(new_spectra_file, selected_bands)

## 烟梗标签生成
这会将纯烟梗图片中识别为杂质的部分提取出来

In [4]:
x_list, y_list = [], []
if (new_label_file is None) and (target_class == 1):
    x_list, y_list = generate_tobacco_label(data, model_file, blk_sz, selected_bands)

## 其他类别杂质阈值分割
通过阈值分割的形式获取其他类别的杂质

In [5]:
if (new_label_file is None) and (target_class != 1):
    img = generate_impurity_label(data, light_threshold, color_dict,
                                  target_class_right=target_class_right,
                                  target_class_left=target_class_left,
                                  split_line=split_line)
    root, _ = os.path.splitext(new_dataset_file)
    cv2.imwrite(root+"_generated.bmp", img)

## 读取标签

In [6]:
if new_label_file is not None:
    label = cv2.imread(new_label_file)
    print(label.shape)
    x_list, y_list = split_xy(data, label, blk_sz, sensitivity=sensitivity, color_dict=color_dict, add_background=add_background)

(600, 1024, 3)


In [7]:
print(len(x_list), len(y_list))

148 148


## 读取旧数据合并

In [8]:
with open(dataset_file, 'rb') as f:
    x, y = pickle.load(f)
x.extend(x_list)
y.extend(y_list)
with open(new_dataset_file, 'wb') as f:
    pickle.dump((x, y), f)

## 批量数据的处理